# Feature importance 평가를 위한 SHAP
- kfold validation 실행 후 SHAP 
- 연산에 소요되는 시간 확인을 위해 우선적으로 catboost model에 대해서만 실행

In [ ]:
혹시 연구실 컴퓨터에 shap 설치 안되어 있으면 설치해서 실행하길...

In [1]:
import pandas as pd
import shap
import numpy as np
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold

ModuleNotFoundError: No module named 'shap'

In [ ]:
df = pd.read_csv("dataset.csv")
X = df[['cement', 'slag', 'fly_ash', 'water',
        'superplasticizer', 'coarse_aggregate', 'fine_aggregate', 'age']]
y = df['CCS']

In [ ]:
# 5-fold cross-validation 실행
kf = KFold(n_splits=5, shuffle=True, random_state=42)
shap_values_all = []
feature_names = X.columns

In [ ]:
# Catboost model training + SHAP 값 계산 (fold 별 test set 기준)
for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"Fold {fold+1}...")
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # CatBoost 모델 학습
    model = CatBoostRegressor(verbose=0, random_state=42)
    model.fit(X_train, y_train)

    # SHAP 값 계산 
    explainer = shap.Explainer(model, X_test)
    shap_values = explainer(X_test)

    # 절댓값 평균을 위해 저장
    shap_values_all.append(abs(shap_values.values))

In [ ]:
# SHAP 평균값 계산 -> 최종 importance 계산
mean_shap = np.mean(np.array(shap_values_all), axis=0)  # shape: (n_samples, n_features)
feature_importance = pd.Series(mean_shap.mean(axis=0), index=feature_names).sort_values(ascending=False)


In [ ]:
print("\n Feature Importance (Mean SHAP Values across 5 folds):")
print(feature_importance)